In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_json('recipes.json', lines=True)

In [3]:
df.head()

rate author_tip      budget prep_time  \
0    5             bon marché    15 min   
1    3             Coût moyen       2 h   
2  4.6             bon marché    15 min   
3    5             bon marché    45 min   
4    5             Coût moyen    30 min   

                                         ingredients  \
0  [600g de pâte à crêpe, 1/2 orange, 1/2 banane,...   
1  [1kg d'agneau, 300g de riz, 24 abricots secs, ...   
2  [1kg d'épaule d'agneau désossée, 3 oignons, 2 ...   
3  [2 aubergines, 3 tomates, 700g d'épaule d'agne...   
4  [1 gigot d'agneau, 2 oignons, 6gousses d'ail, ...   

                                                name           author  \
0  6 ingrédients que l’on peut ajouter sur une cr...          Nutella   
1                Agneau à l'abricot (recette turque)   oscar_15167769   
2                                Agneau à l'oriental         zoreille   
3                Agneau à la grec (aubergine & feta)      richard_148   
4                  Agneau à la mangue et aux amandes  roxane_16871304   

     difficulty people_quantity cook_time  \
0   très facile               6    10 min   
1  Niveau moyen               6       1 h   
2   très facile               6       1 h   
3        facile               4       1 h   
4   très facile               4       1 h   

                                           tags total_time  \
0  [Crêpe, Crêpes sucrées, Végétarien, Dessert]     25 min   
1                              [Plat principal]        3 h   
2                 [Sans gluten, Plat principal]       1h15   
3                 [Sans gluten, Plat principal]       1h45   
4                 [Sans gluten, Plat principal]       1h30   

                                               image  nb_comments  
0  https://assets.afcdn.com/recipe/20171006/72810...            1  
1                                                               1  
2                                                              10  
3                                                               6  
4                                                               2

In [4]:
df['name'] = df['name'].str.capitalize()

In [5]:
df['difficulty'].value_counts()

facile          4796
très facile     3942
Niveau moyen     719
difficile         34
Name: difficulty, dtype: int64

In [6]:
df['difficulty'] = df['difficulty'].str.lower()

In [7]:
df['budget'].value_counts()

bon marché    5907
Coût moyen    3256
assez cher     328
Name: budget, dtype: int64

In [8]:
df['budget'] = df['budget'].str.lower()

In [9]:
(df['image'].values != '').sum()

4085

In [10]:
(df['author_tip'].values != '').sum()

0

In [11]:
df['rate'].value_counts()

       2202
5      1511
4       975
4.8     710
4.7     647
4.5     645
4.3     488
4.6     388
4.4     365
4.9     320
3       263
4.2     169
3.5     164
2        97
3.8      95
3.7      88
4.1      81
3.3      55
1        50
3.9      39
3.6      31
2.5      29
3.4      20
3.2      16
0        11
2.8      11
2.7       7
2.3       7
3.1       2
2.6       2
2.4       1
1.5       1
1.7       1
Name: rate, dtype: int64

In [12]:
df['rate'] = df['rate'].replace('', np.nan, regex=True)
df['rate'] = df['rate'].astype(float)

In [13]:
df['people_quantity'].value_counts()

4      5256
6      1824
2      1319
8       320
3       226
5       211
1       176
10       65
12       33
20       12
          8
24        7
7         5
30        4
40        4
15        4
14        3
25        3
9         3
16        3
500       2
100       1
18        1
50        1
Name: people_quantity, dtype: int64

In [14]:
df['people_quantity'] = df['people_quantity'].replace('', 0, regex=True)
df['people_quantity'] = df['people_quantity'].astype(int)

In [15]:
from datetime import timedelta
import re

UNITS = {'m':'minutes', 'h':'hours', 'j':'days'}

def timeparse(s):
    s = s.replace('min', 'm').replace(' ', '')
    return int(timedelta(**{
        UNITS.get(m.group('unit').lower(), 'minutes'): float(m.group('val'))
        for m in re.finditer(r'(?P<val>\d+(\.\d+)?)(?P<unit>[minhj]?)', s, flags=re.I)
    }).total_seconds()/60)

df['prep_time_in_minutes'] = df['prep_time'].apply(timeparse)
df['cook_time_in_minutes'] = df['cook_time'].apply(timeparse)
df['total_time_in_minutes'] = df['total_time'].apply(timeparse)

df.groupby('total_time').agg({'total_time_in_minutes': ['max']})

total_time_in_minutes
                                  max
total_time                           
                                    0
1 h                                60
1 j                              1440
1 j 1 h 30 min                   1530
1 j 11 h 20 min                  2120
...                               ...
7h30                              450
7h35                              455
8 h                               480
8 min                               8
9 min                               9

[186 rows x 1 columns]

In [16]:
df['prep_time_in_minutes'] = df['prep_time_in_minutes'].replace('', 0, regex=True)
df['prep_time_in_minutes'] = df['prep_time_in_minutes'].astype(int)

df['cook_time_in_minutes'] = df['cook_time_in_minutes'].replace('', 0, regex=True)
df['cook_time_in_minutes'] = df['cook_time_in_minutes'].astype(int)

df['total_time_in_minutes'] = df['total_time_in_minutes'].replace('', 0, regex=True)
df['total_time_in_minutes'] = df['total_time_in_minutes'].astype(int)

In [17]:
df.explode('ingredients')['ingredients'].value_counts()

Sel                                                 5519
Poivre                                              5135
1 oignon                                            1775
Huile d'olive                                       1239
2 oignons                                            838
                                                    ... 
150g de pâte feuilletée                                1
2 avocats coupés en deux puis en tranches fines.       1
2cubes de Bouillon KUB Volaille MAGGI®                 1
30g de mayonnaise                                      1
2cuillères à café de ketchup                           1
Name: ingredients, Length: 27867, dtype: int64

In [18]:
df.explode('tags')['tags'].value_counts()

Plat principal               9292
Sans gluten                  4424
Viande                       1909
Végétarien                   1642
Pâtes, riz, semoule          1358
                             ... 
Palets et langues de chat       1
Recettes d'Afrique              1
Apéro léger                     1
Cake au citron                  1
Pissaladière                    1
Name: tags, Length: 445, dtype: int64

In [19]:
df['author'].value_counts()

Anonyme              633
Marmiton_Recettes    104
Herta                 87
Nutella               67
colorado              61
                    ... 
lorraine               1
romy_14611896          1
romy_13979912          1
Jacky                  1
camila_16414613        1
Name: author, Length: 6399, dtype: int64

In [20]:
df['number_of_comments'] = df['nb_comments']
df['id'] = df.index + 1
df = df[['id','name','rate','people_quantity','prep_time_in_minutes','cook_time_in_minutes','total_time_in_minutes',
         'image','number_of_comments','author','difficulty','budget','ingredients','tags']]

In [21]:
df = df.replace('', np.nan, regex=True)

In [22]:
if not os.path.exists('tables'):
    os.makedirs('tables')

In [23]:
df = df.assign(author_id=df.author.factorize()[0] + 1)
df_authors = df[['author_id', 'author']].drop_duplicates()
df_authors.columns = ['id', 'name']
df_authors.to_csv('tables/authors.csv', index=False, encoding='utf-8')

In [24]:
df = df.assign(difficulty_id=df.difficulty.factorize()[0] + 1)
df_difficulties = df[['difficulty_id', 'difficulty']].drop_duplicates()
df_difficulties.columns = ['id', 'name']
df_difficulties.to_csv('tables/difficulties.csv', index=False, encoding='utf-8')

In [25]:
df = df.assign(budget_id=df.budget.factorize()[0] + 1)
df_budgets = df[['budget_id', 'budget']].drop_duplicates()
df_budgets.columns = ['id', 'name']
df_budgets.to_csv('tables/budgets.csv', index=False, encoding='utf-8')

In [26]:
df_recipes = df[['id','name','rate','people_quantity','prep_time_in_minutes','cook_time_in_minutes','total_time_in_minutes', 
                 'image','number_of_comments','author_id','difficulty_id','budget_id']]
df_recipes.to_csv('tables/recipes.csv', index=False, encoding='utf-8')

In [27]:
df_recipes_ingredients = df.explode('ingredients')
df_recipes_ingredients = df_recipes_ingredients.assign(ingredient_id=df_recipes_ingredients.ingredients.factorize()[0] + 1)

df_ingredients = df_recipes_ingredients[['ingredient_id', 'ingredients']].drop_duplicates()
df_ingredients.columns = ['id', 'name']
df_ingredients.to_csv('tables/ingredients.csv', index=False, encoding='utf-8')

df_recipes_ingredients = df_recipes_ingredients[['id', 'ingredient_id']]
df_recipes_ingredients.columns = ['recipe_id', 'ingredient_id']
df_recipes_ingredients.to_csv('tables/recipes_ingredients.csv', index=False, encoding='utf-8')

In [28]:
df_recipes_tags = df.explode('tags')
df_recipes_tags = df_recipes_tags.assign(tag_id=df_recipes_tags.tags.factorize()[0] + 1)

df_tags = df_recipes_tags[['tag_id', 'tags']].drop_duplicates()
df_tags.columns = ['id', 'name']
df_tags.to_csv('tables/tags.csv', index=False, encoding='utf-8')

df_recipes_tags = df_recipes_tags[['id', 'tag_id']]
df_recipes_tags.columns = ['recipe_id', 'tag_id']
df_recipes_tags.to_csv('tables/recipes_tags.csv', index=False, encoding='utf-8')